# Ramsey King
# DSC 550 - Data Mining
# September 11, 2021
# Week 2: Handling Categorical Data, Text, Dates & Times
# Exercise 2.2 


You can find the dataset controversial-comments.jsonl for this exercise in the Weekly Resources: Week 2 Data Files.
Pre-processing Text: For this part, you will start by reading the controversial-comments.jsonl file into a DataFrame.

In [89]:
import pandas as pd
import numpy as np

comments_df = pd.read_json('controversial-comments.jsonl', lines=True)
comments_df.head()
# I am going to limit the number of entries from 950,000 to 50,000 as per the instructions in the assignment to help with the runtime.

comments_df = comments_df[:50000].copy()


A. Convert all text to lowercase letters.

In [90]:
comments_df['txt'] = comments_df['txt'].apply(str.lower)
comments_df['txt'][:5]

0    well it's great that he did something about th...
1                         you are right mr. president.
2    you have given no input apart from saying i am...
3    i get the frustration but the reason they want...
4    i am far from an expert on tpp and i would ten...
Name: txt, dtype: object

B. Remove all punctuation from the text.


In [92]:
import re
comments_df['txt'] = comments_df['txt'].apply(lambda remove_punct: re.sub(r'[^\w\s]', '', remove_punct))
comments_df['txt'][:5]

0    well its great that he did something about tho...
1                           you are right mr president
2    you have given no input apart from saying i am...
3    i get the frustration but the reason they want...
4    i am far from an expert on tpp and i would ten...
Name: txt, dtype: object

C. Remove stop words.


In [95]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = stopwords.words('english')

def rem_stop_words(rem_list):
    return [word for word in rem_list if word not in stop_words]

comments_df['txt_tokenized'] = comments_df['txt'].apply(word_tokenize)
comments_df['txt_tokenized'] = comments_df['txt_tokenized'].apply(rem_stop_words)
comments_df['txt_tokenized'][:5]

0    [well, great, something, beliefs, office, doub...
1                               [right, mr, president]
2    [given, input, apart, saying, wrong, argument,...
3    [get, frustration, reason, want, way, foundati...
4    [far, expert, tpp, would, tend, agree, lot, pr...
Name: txt_tokenized, dtype: object

D. Apply NLTK’s PorterStemmer.

In [96]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def apply_porter_stemmer(tokenized_words):
    return [porter.stem(word) for word in tokenized_words]

comments_df['txt_tokenized_stemmed'] = comments_df['txt_tokenized'].apply(apply_porter_stemmer)
comments_df['txt_tokenized_stemmed'][:5]

0    [well, great, someth, belief, offic, doubt, tr...
1                                  [right, mr, presid]
2    [given, input, apart, say, wrong, argument, cl...
3    [get, frustrat, reason, want, way, foundat, co...
4    [far, expert, tpp, would, tend, agre, lot, pro...
Name: txt_tokenized_stemmed, dtype: object

Now that the data is pre-processed, you will apply three different techniques to get it into a usable form for
model-building. Apply each of the following steps (individually) to the pre-processed data.

A. Convert each text entry into a word-count vector (see sects 5.3 & 6.8 in the Machine Learning with Python Cookbook).

In [98]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
results = count.fit_transform(comments_df['txt'])
print(comments_df.info())
print(type(results))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   con                    50000 non-null  int64 
 1   txt                    50000 non-null  object
 2   txt_tokenized          50000 non-null  object
 3   txt_tokenized_stemmed  50000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB
None
<class 'scipy.sparse.csr.csr_matrix'>


B. Convert each text entry into a part-of-speech tag vector (see section 6.7 in the Machine Learning with Python Cookbook).

In [103]:
from nltk import pos_tag
from nltk import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
# nltk.download('averaged_perceptron_tagger')
tweets = comments_df['txt'].tolist()
tagged_tweets = []
for tweet in tweets:
    tweet_tag = nltk.pos_tag(word_tokenize(tweet))
    tagged_tweets.append([tag for word, tag in tweet_tag])

one_hot_multi = MultiLabelBinarizer()
one_hot_multi.fit_transform(tagged_tweets)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

C. Convert each entry into a term frequency-inverse document frequency (tfidf) vector
(see section 6.9 in the Machine Learning with Python Cookbook).

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_data = np.array(tweets)
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(text_data)
tfidf.vocabulary_

{'well': 44422,
 'its': 23151,
 'great': 17239,
 'that': 40436,
 'he': 18843,
 'did': 11748,
 'something': 37622,
 'about': 2241,
 'those': 40799,
 'beliefs': 5395,
 'while': 44590,
 'was': 44189,
 'in': 21907,
 'office': 28858,
 'doubt': 12639,
 'trump': 41724,
 'would': 45102,
 'fight': 15230,
 'the': 40463,
 'un': 42388,
 'for': 15719,
 'it': 23095,
 'so': 37469,
 'im': 21670,
 'really': 33304,
 'happy': 18646,
 'obama': 28647,
 'couldoh': 9895,
 'wait': 44032,
 'you': 45469,
 'are': 4066,
 'right': 34700,
 'mr': 27257,
 'president': 31765,
 'have': 18794,
 'given': 16783,
 'no': 28060,
 'input': 22436,
 'apart': 3849,
 'from': 16108,
 'saying': 35589,
 'am': 3253,
 'wrong': 45178,
 'argument': 4104,
 'clearly': 8361,
 'get': 16683,
 'frustration': 16148,
 'but': 6910,
 'reason': 33344,
 'they': 40605,
 'want': 44121,
 'them': 40506,
 'to': 41076,
 'do': 12407,
 'way': 44274,
 'is': 23001,
 'because': 5287,
 'foundation': 15891,
 'more': 27107,
 'complex': 9040,
 'problems': 32007,


For the three techniques in problem (2) above, give an example where each would be useful.

Word count vector - can help identify a subject of a text and a hidden subjet within text.  Can also be used for sentiment analysis - THere are references out there that use sentiment analysis to their benefit.  One example would be an airline company that reads comments from Twitter.  If the airline company identifies certain words that have a negative sentiment, they can look out for these words and make marketing adjustments as necessary to help with customer satisfaction.

Part of speech tag vector - Tagging parts of speech is important in that it allows for the context of a word to be identified.  For example, if we continue with the twitter comment airline example, the word "late" without any context would probably be viewed as negative when it applies to an airline company.  For example, if a comment was " #airlinecompany flight 1234 is running late as usual!" then the word "late" in this context has a negative sentiment.  However, if another comment was "#airlinecompany made sure that I made my flight even though I got to the airport really late.", then that can be viewed positively.

Term frequency - inverse document frequency - One of the most important uses of term frequency - inverse document frequency is found in something we use almost every day, Google search.  When we search for a term in Google, the words we search are compared with all the text that Google is able to search in its universe.  Those terms are compared with the texts Google searches upon, and by using the Term Frequency - Inverse Document Frequency, it is able to return the most relevant documents (websites) to our terms.